# This Notebook contains the BindCraft outputs to Boltz refolding pipeline

## Step 1: Format BindCraft outputs into a file suitable for downstream Boltz refolding

In [2]:
## Default python libraries
import os
import shutil
import glob
import os
import json
import re

from collections import defaultdict
from pathlib import Path
from collections import Counter

## External libraries
import pandas as pd
import numpy as np
import tqdm

pd.set_option('display.max_columns', 500)

### 1.1 Load the bindcraft output

In [3]:
## Replace the output paths with your own
bindcraft_final_design_stats_paths = [
    "/data1/lareauc/users/chuh/softwares/CARPNN/examples/00_bindcraft_outputs/BCMA_run_1/final_design_stats.csv",
    "/data1/lareauc/users/chuh/softwares/CARPNN/examples/00_bindcraft_outputs/BCMA_run_2/final_design_stats.csv",
    ## Add more here if needed
]
bc_df_combined = pd.concat([pd.read_csv(path) for path in bindcraft_final_design_stats_paths],ignore_index=True)
bc_df_combined

,Rank,Design,Protocol,Length,Seed,Helicity,Target_Hotspot,Sequence,InterfaceResidues,MPNN_score,MPNN_seq_recovery,Average_pLDDT,1_pLDDT,2_pLDDT,3_pLDDT,4_pLDDT,5_pLDDT,Average_pTM,1_pTM,2_pTM,3_pTM,4_pTM,5_pTM,Average_i_pTM,1_i_pTM,2_i_pTM,3_i_pTM,4_i_pTM,5_i_pTM,Average_pAE,1_pAE,2_pAE,3_pAE,4_pAE,5_pAE,Average_i_pAE,1_i_pAE,2_i_pAE,3_i_pAE,4_i_pAE,5_i_pAE,Average_i_pLDDT,1_i_pLDDT,2_i_pLDDT,3_i_pLDDT,4_i_pLDDT,5_i_pLDDT,Average_ss_pLDDT,1_ss_pLDDT,2_ss_pLDDT,3_ss_pLDDT,4_ss_pLDDT,5_ss_pLDDT,Average_Unrelaxed_Clashes,1_Unrelaxed_Clashes,2_Unrelaxed_Clashes,3_Unrelaxed_Clashes,4_Unrelaxed_Clashes,5_Unrelaxed_Clashes,Average_Relaxed_Clashes,1_Relaxed_Clashes,2_Relaxed_Clashes,3_Relaxed_Clashes,4_Relaxed_Clashes,5_Relaxed_Clashes,Average_Binder_Energy_Score,1_Binder_Energy_Score,2_Binder_Energy_Score,3_Binder_Energy_Score,4_Binder_Energy_Score,5_Binder_Energy_Score,Average_Surface_Hydrophobicity,1_Surface_Hydrophobicity,2_Surface_Hydrophobicity,3_Surface_Hydrophobicity,4_Surface_Hydrophobicity,5_Surface_Hydrophobicity,Average_ShapeComplementarity,1_ShapeComplementarity,2_ShapeComplementarity,3_ShapeComplementarity,4_ShapeComplementarity,5_ShapeComplementarity,Average_PackStat,1_PackStat,2_PackStat,3_PackStat,4_PackStat,5_PackStat,Average_dG,1_dG,2_dG,3_dG,4_dG,5_dG,Average_dSASA,1_dSASA,2_dSASA,3_dSASA,4_dSASA,5_dSASA,Average_dG/dSASA,1_dG/dSASA,2_dG/dSASA,3_dG/dSASA,4_dG/dSASA,5_dG/dSASA,Average_Interface_SASA_%,1_Interface_SASA_%,2_Interface_SASA_%,3_Interface_SASA_%,4_Interface_SASA_%,5_Interface_SASA_%,Average_Interface_Hydrophobicity,1_Interface_Hydrophobicity,2_Interface_Hydrophobicity,3_Interface_Hydrophobicity,4_Interface_Hydrophobicity,5_Interface_Hydrophobicity,Average_n_InterfaceResidues,1_n_InterfaceResidues,2_n_InterfaceResidues,3_n_InterfaceResidues,4_n_InterfaceResidues,5_n_InterfaceResidues,Average_n_InterfaceHbonds,1_n_InterfaceHbonds,2_n_InterfaceHbonds,3_n_InterfaceHbonds,4_n_InterfaceHbonds,5_n_InterfaceHbonds,Average_InterfaceHbondsPercentage,1_InterfaceHbondsPercentage,2_InterfaceHbondsPercentage,3_InterfaceHbondsPercentage,4_InterfaceHbondsPercentage,5_InterfaceHbondsPercentage,Average_n_InterfaceUnsatHbonds,1_n_InterfaceUnsatHbonds,2_n_InterfaceUnsatHbonds,3_n_InterfaceUnsatHbonds,4_n_InterfaceUnsatHbonds,5_n_InterfaceUnsatHbonds,Average_InterfaceUnsatHbondsPercentage,1_InterfaceUnsatHbondsPercentage,2_InterfaceUnsatHbondsPercentage,3_InterfaceUnsatHbondsPercentage,4_InterfaceUnsatHbondsPercentage,5_InterfaceUnsatHbondsPercentage,Average_Interface_Helix%,1_Interface_Helix%,2_Interface_Helix%,3_Interface_Helix%,4_Interface_Helix%,5_Interface_Helix%,Average_Interface_BetaSheet%,1_Interface_BetaSheet%,2_Interface_BetaSheet%,3_Interface_BetaSheet%,4_Interface_BetaSheet%,5_Interface_BetaSheet%,Average_Interface_Loop%,1_Interface_Loop%,2_Interface_Loop%,3_Interface_Loop%,4_Interface_Loop%,5_Interface_Loop%,Average_Binder_Helix%,1_Binder_Helix%,2_Binder_Helix%,3_Binder_Helix%,4_Binder_Helix%,5_Binder_Helix%,Average_Binder_BetaSheet%,1_Binder_BetaSheet%,2_Binder_BetaSheet%,3_Binder_BetaSheet%,4_Binder_BetaSheet%,5_Binder_BetaSheet%,Average_Binder_Loop%,1_Binder_Loop%,2_Binder_Loop%,3_Binder_Loop%,4_Binder_Loop%,5_Binder_Loop%,Average_InterfaceAAs,1_InterfaceAAs,2_InterfaceAAs,3_InterfaceAAs,4_InterfaceAAs,5_InterfaceAAs,Average_Hotspot_RMSD,1_Hotspot_RMSD,2_Hotspot_RMSD,3_Hotspot_RMSD,4_Hotspot_RMSD,5_Hotspot_RMSD,Average_Target_RMSD,1_Target_RMSD,2_Target_RMSD,3_Target_RMSD,4_Target_RMSD,5_Target_RMSD,Average_Binder_pLDDT,1_Binder_pLDDT,2_Binder_pLDDT,3_Binder_pLDDT,4_Binder_pLDDT,5_Binder_pLDDT,Average_Binder_pTM,1_Binder_pTM,2_Binder_pTM,3_Binder_pTM,4_Binder_pTM,5_Binder_pTM,Average_Binder_pAE,1_Binder_pAE,2_Binder_pAE,3_Binder_pAE,4_Binder_pAE,5_Binder_pAE,Average_Binder_RMSD,1_Binder_RMSD,2_Binder_RMSD,3_Binder_RMSD,4_Binder_RMSD,5_Binder_RMSD,DesignTime,Notes,TargetSettings,Filters,AdvancedSettings
0,1,BCMA_l64_s257143_mpnn2,4stage,64,257143,-0.3,NaN,KLSKEEQRKWIEENKEKVLKVVLSSEHKEWYAGMSWEELKKIFDYV...,"B21,B

### 1.2 Make a colabfold style csv with the target sequence of interest

In [4]:
## Replace target_name and target_seq with your own.
target_name = "BCMA"
target_seq = "CSQNEYFDSLLHACIPCQLRCSSNTPPLTCQRYCNA" # We use the ECD portion only

def bc_to_colab(bc_df,name,seq):
    new_rows = []
    for idx,row in bc_df.iterrows():
        new_rows.append({
            "id":f"{row['Design']}__{name}", ## The double underscore (__) delimiter is important here
            "sequence":f"{row['Sequence']}:{seq}" ## The colon (:) delimiter is important here
        })
    colab_df = pd.DataFrame(new_rows)
    return colab_df

colab_df = bc_to_colab(bc_df_combined,target_name,target_seq)
colab_df

,id,sequence
0,BCMA_l64_s257143_mpnn2__BCMA,KLSKEEQRKWIEENKEKVLKVVLSSEHKEWYAGMSWEELKKIFDYV...
1,BCMA_l64_s257143_mpnn10__BCMA,MLSKEEQRKWIEENKEKVMKVVLSSEHAEWYAGMSWEELKKIFDYV...
2,BCMA_l54_s17866_mpnn3__BCMA,SISDMSVEELEREVEAWEQIARWTEENGMPQWTVERAREMVELYRK...
3,BCMA_l58_s790689_mpnn2__BCMA,SSIEKEIQEEIDIFKYLLSQGMDPEHIKEGIIRRARNEPWFSEEEI...
4,BCMA_l54_s662851_mpnn3__BCMA,SMEEYQKKMSEFANEITEEAMKDPEKYKEVSWMLNTEEGRYKLKEI...
5,BCMA_l61_s64905_mpnn4__BCMA,SEKEMEELKKRFEEVREEARRVWEEKERHREDEEFVEELERRMEEL...
6,BCMA_l57_s933667_mpnn1__BCMA,SWREMSPREYVLEVGVERLREIIREHGGRDSPRFQEFERQFRHWYG...
7,BCMA_l54_s398596_mpnn9__BCMA,MKISEGEAHKRFMEIKEEFRRKRKEAEPLGEEEIEKLREEALKEVK...
8,BCMA_l54_s398596_mpnn17__BCMA,MKISEGEAHQKFMEIKEEFRRKRKEAEPLGEEEIEKLREEALEKVK...
9,BCMA_l55_s835555_mpnn4__BCMA,SMLEKTKEELKKQIEMGEKETVYRVVSYLVEHPERSMMSPEEIEEI...


In [14]:
## Write the csv
## Replace the colab_write_path with your own
colab_write_path = "/data1/lareauc/users/chuh/softwares/CARPNN/examples/01_bindcraft_boltz_refolded/input_tables/colabfold_table.csv"

os.makedirs(os.path.dirname(colab_write_path), exist_ok=True)
colab_df.to_csv(colab_write_path,index=False)

## Step 2: Run the Boltz Prediction Pipeline to Obtain Predicted Structures
See the README in the Boltz workflow for more details on how to set it up, but the rough pipeline is:
1. **Run the boltz prediction pipeline to obtain predicted structures and metrics** (`sbatch workflows/Boltz/boltz_pipeline_wrapper_step_02_05_HC.sh <path to colabfold_table.csv> <prediction output directory> <number of jobs to divide into>`)
2. **Calculate ipSAE / pAE interaction / rosetta-based metrics based on the  Boltz outputs** (`sbatch workflows/Boltz/boltz_pipeline_wrapper_step_06_08_HC <boltz prediction output directory>`)
3. **Run the output aggregation script to summarize the outputs per directory** (`sbatch workflows/Boltz/10_aggregate_boltz_outputs.sh <prediction output directory> <aggregated output directory>`)

Example commands:

1. `sbatch examples/01_bindcraft_boltz_refolded/input_tables/colabfold_table.csv examples/01_bindcraft_boltz_refolded/predictions 2`
2. `sbatch workflows/Boltz/boltz_pipeline_wrapper_step_06_08_HC.sh examples/01_bindcraft_boltz_refolded/predictions/boltz_predictions`
3. `sbatch workflows/Boltz/10_aggregate_boltz_outputs.sh examples/01_bindcraft_boltz_refolded/predictions/boltz_predictions examples/01_bindcraft_boltz_refolded/predictions/boltz_predictions_aggregated`

## Step 3: Load the aggregated outputs

### 3.1 Load the aggregated outputs and add sequence information back into the dataframes

In [12]:
## Replace the aggregation directory with the one specified during the run
agg_dir = "/data1/lareauc/users/chuh/softwares/CARPNN/examples/01_bindcraft_boltz_refolded/predictions/boltz_predictions_aggregated"

combined_boltz_df = pd.concat([pd.read_csv(path) for path in glob.glob(f"{agg_dir}/*.csv")],ignore_index=True)
combined_boltz_df

,prediction_name,pdb_model_name,pdb_path,confidence_path,pae_path,plddt_path,confidence_score,ptm,iptm,ligand_iptm,protein_iptm,complex_plddt,complex_iplddt,complex_pde,complex_ipde,chains_ptm_0,chains_ptm_1,pair_chains_iptm_0_0,pair_chains_iptm_0_1,pair_chains_iptm_1_0,pair_chains_iptm_1_1,pae_all,pae_chain_a,pae_chain_b,pae_interaction,Chn1,Chn2,PAE,Dist,Type,ipSAE,ipSAE_d0chn,ipSAE_d0dom,ipTM_af,ipTM_d0chn,pDockQ,pDockQ2,LIS,n0res,n0chn,n0dom,d0res,d0chn,d0dom,nres1,nres2,dist1,dist2,Model,i_pLDDT,ss_pLDDT,Unrelaxed_Clashes,Relaxed_Clashes,Binder_Energy_Score,Surface_Hydrophobicity,ShapeComplementarity,PackStat,dG,dSASA,dG/dSASA,Interface_SASA_%,Interface_Hydrophobicity,n_InterfaceResidues,n_InterfaceHbonds,InterfaceHbondsPercentage,n_InterfaceUnsatHbonds,InterfaceUnsatHbondsPercentage,InterfaceAAs,Interface_Helix%,Interface_BetaSheet%,Interface_Loop%,Binder_Helix%,Binder_BetaSheet%,Binder_Loop%,binder_path,binder_relaxed_path
0,BCMA_l61_s64905_mpnn4__BCMA,BCMA_l61_s64905_mpnn4__BCMA_model_2,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.936145,0.955550,0.945506,0.0,0.945506,0.933805,0.941414,0.278135,0.340065,0.965639,0.965048,0.965639,0.923435,0.945506,0.965048,1.522581,1.169044,1.158353,1.929582,A,B,10,10,max,0.883050,0.950799,0.950799,0.946,0.950799,0.3669,0.8799,0.8408,61,97,97,2.64,3.59,3.59,61,36,26,16,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.96,0.97,0,0,-170.26,0.20,0.67,0.68,-41.56,1425.31,-2.92,31.27,66.67,18,6,33.33,1.0,5.56,"{'A': 1, 'C': 0, 'D': 0, 'E': 1, 'F': 1, 'G': ...",100.00,0.0,0.00,90.16,0.0,9.84,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
1,BCMA_l61_s64905_mpnn4__BCMA,BCMA_l61_s64905_mpnn4__BCMA_model_0,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.937535,0.951433,0.939903,0.0,0.939903,0.936943,0.947970,0.288089,0.367192,0.963098,0.962673,0.963098,0.923888,0.939903,0.962673,1.662470,1.304377,1.240707,2.090309,A,B,10,10,max,0.881840,0.943222,0.943222,0.940,0.943222,0.3881,0.8957,0.8284,61,97,97,2.64,3.59,3.59,61,36,28,17,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.97,0.97,2,0,-168.24,0.20,0.72,0.64,-41.22,1408.31,-2.93,31.00,66.67,18,6,33.33,1.0,5.56,"{'A': 1, 'C': 0, 'D': 0, 'E': 1, 'F': 1, 'G': ...",100.00,0.0,0.00,90.16,0.0,9.84,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
2,BCMA_l61_s64905_mpnn4__BCMA,BCMA_l61_s64905_mpnn4__BCMA_model_3,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.935590,0.947600,0.933816,0.0,0.933816,0.936033,0.948799,0.296900,0.399565,0.962050,0.957838,0.962050,0.923114,0.933816,0.957838,1.754216,1.332538,1.355371,2.229163,A,B,10,10,max,0.878208,0.934759,0.934759,0.934,0.934759,0.3403,0.8992,0.8219,61,97,97,2.64,3.59,3.59,61,36,28,16,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.97,0.97,1,0,-176.05,0.23,0.73,0.65,-38.47,1371.93,-2.80,30.77,66.67,18,3,16.67,0.0,0.00,"{'A': 1, 'C': 0, 'D': 0, 'E': 1, 'F': 1, 'G': ...",100.00,0.0,0.00,90.16,0.0,9.84,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
3,BCMA_l61_s64905_mpnn4__BCMA,BCMA_l61_s64905_mpnn4__BCMA_model_4,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.935115,0.945381,0.929671,0.0,0.929671,0.936476,0.950363,0.297723,0.406137,0.960503,0.958803,0.960503,0.921000,0.929671,0.958803,1.778701,1.358362,1.349158,2.261571,A,B,10,10,max,0.8747

### 3.2 Add sequence information back to the prediction dataframe

In [13]:
## This is the input file used for the boltz reprediction 
## Replace the input path with the one provided to the boltz pipeline
colab_input_path = "/data1/lareauc/users/chuh/softwares/CARPNN/examples/01_bindcraft_boltz_refolded/input_tables/colabfold_table.csv"

colab_input_df = pd.read_csv(colab_input_path)
colab_input_df["chain_a_seq"] = colab_input_df["sequence"].apply(lambda x: x.split(":")[0])
colab_input_df["chain_b_seq"] = colab_input_df["sequence"].apply(lambda x: x.split(":")[1])
combined_boltz_df = pd.concat(pd.read_csv(path) for path in glob.glob(f"{agg_dir}/*.csv"))
combined_boltz_df = combined_boltz_df.merge(colab_input_df[["id","chain_a_seq","chain_b_seq"]],left_on="prediction_name",right_on="id",how="left")
combined_boltz_df

,prediction_name,pdb_model_name,pdb_path,confidence_path,pae_path,plddt_path,confidence_score,ptm,iptm,ligand_iptm,protein_iptm,complex_plddt,complex_iplddt,complex_pde,complex_ipde,chains_ptm_0,chains_ptm_1,pair_chains_iptm_0_0,pair_chains_iptm_0_1,pair_chains_iptm_1_0,pair_chains_iptm_1_1,pae_all,pae_chain_a,pae_chain_b,pae_interaction,Chn1,Chn2,PAE,Dist,Type,ipSAE,ipSAE_d0chn,ipSAE_d0dom,ipTM_af,ipTM_d0chn,pDockQ,pDockQ2,LIS,n0res,n0chn,n0dom,d0res,d0chn,d0dom,nres1,nres2,dist1,dist2,Model,i_pLDDT,ss_pLDDT,Unrelaxed_Clashes,Relaxed_Clashes,Binder_Energy_Score,Surface_Hydrophobicity,ShapeComplementarity,PackStat,dG,dSASA,dG/dSASA,Interface_SASA_%,Interface_Hydrophobicity,n_InterfaceResidues,n_InterfaceHbonds,InterfaceHbondsPercentage,n_InterfaceUnsatHbonds,InterfaceUnsatHbondsPercentage,InterfaceAAs,Interface_Helix%,Interface_BetaSheet%,Interface_Loop%,Binder_Helix%,Binder_BetaSheet%,Binder_Loop%,binder_path,binder_relaxed_path,id,chain_a_seq,chain_b_seq
0,BCMA_l61_s64905_mpnn4__BCMA,BCMA_l61_s64905_mpnn4__BCMA_model_2,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.936145,0.955550,0.945506,0.0,0.945506,0.933805,0.941414,0.278135,0.340065,0.965639,0.965048,0.965639,0.923435,0.945506,0.965048,1.522581,1.169044,1.158353,1.929582,A,B,10,10,max,0.883050,0.950799,0.950799,0.946,0.950799,0.3669,0.8799,0.8408,61,97,97,2.64,3.59,3.59,61,36,26,16,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.96,0.97,0,0,-170.26,0.20,0.67,0.68,-41.56,1425.31,-2.92,31.27,66.67,18,6,33.33,1.0,5.56,"{'A': 1, 'C': 0, 'D': 0, 'E': 1, 'F': 1, 'G': ...",100.00,0.0,0.00,90.16,0.0,9.84,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,BCMA_l61_s64905_mpnn4__BCMA,SEKEMEELKKRFEEVREEARRVWEEKERHREDEEFVEELERRMEEL...,CSQNEYFDSLLHACIPCQLRCSSNTPPLTCQRYCNA
1,BCMA_l61_s64905_mpnn4__BCMA,BCMA_l61_s64905_mpnn4__BCMA_model_0,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.937535,0.951433,0.939903,0.0,0.939903,0.936943,0.947970,0.288089,0.367192,0.963098,0.962673,0.963098,0.923888,0.939903,0.962673,1.662470,1.304377,1.240707,2.090309,A,B,10,10,max,0.881840,0.943222,0.943222,0.940,0.943222,0.3881,0.8957,0.8284,61,97,97,2.64,3.59,3.59,61,36,28,17,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.97,0.97,2,0,-168.24,0.20,0.72,0.64,-41.22,1408.31,-2.93,31.00,66.67,18,6,33.33,1.0,5.56,"{'A': 1, 'C': 0, 'D': 0, 'E': 1, 'F': 1, 'G': ...",100.00,0.0,0.00,90.16,0.0,9.84,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,BCMA_l61_s64905_mpnn4__BCMA,SEKEMEELKKRFEEVREEARRVWEEKERHREDEEFVEELERRMEEL...,CSQNEYFDSLLHACIPCQLRCSSNTPPLTCQRYCNA
2,BCMA_l61_s64905_mpnn4__BCMA,BCMA_l61_s64905_mpnn4__BCMA_model_3,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.935590,0.947600,0.933816,0.0,0.933816,0.936033,0.948799,0.296900,0.399565,0.962050,0.957838,0.962050,0.923114,0.933816,0.957838,1.754216,1.332538,1.355371,2.229163,A,B,10,10,max,0.878208,0.934759,0.934759,0.934,0.934759,0.3403,0.8992,0.8219,61,97,97,2.64,3.59,3.59,61,36,28,16,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.97,0.97,1,0,-176.05,0.23,0.73,0.65,-38.47,1371.93,-2.80,30.77,66.67,18,3,16.67,0.0,0.00,"{'A': 1, 'C': 0, 'D': 0, 'E': 1, 'F': 1, 'G': ...",100.00,0.0,0.00,90.16,0.0,9.84,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,BCMA_l61_s64905_mpnn4__BCMA,SEKEMEELKKRFEEVREEARRVWEEKERHREDEEFVEELERRMEEL...,CSQNEYFDSLLHACIPCQLRCSSNTPPLTCQRYCNA
3,BCMA_l61_s64905_mpnn4__BCMA,BCMA_l61_s64905_mpnn4__BCMA_model_4,/data1/lareauc

## Step 3b (Optional): Calculate the monomer lDDT / RMSD of the binders
Sometimes binders could have high interaction metrics but have unrealistic shapes that make them insoluable. To check this, we can fold up the binders as monomers and check their monomer lDDT and how different they look compared to how they are in complex via RMSD.

### 3b.1: Create the fasta input required for monomer refolding

In [5]:
## Replace fasta_dir with your own. This is where the fasta files wil be written
fasta_dir = "/data1/lareauc/users/chuh/softwares/CARPNN/examples/01_bindcraft_boltz_refolded/input_tables/monomer_fasta"

## This creates a directory containing monomer fastas
def dataframe_to_fasta_and_split(df, output_dir, id_col="Design", seq_col="Sequence", num_parts=5):
    output_dir = Path(output_dir)
    full_fasta_path = output_dir / "binder_monomer.fasta"
    chunk_dir = output_dir / "fasta_chunked"
    chunk_dir.mkdir(parents=True, exist_ok=True)

    # Write full fasta
    entries = [(f">{row[id_col]}", row[seq_col]) for _, row in df.iterrows()]
    with open(full_fasta_path, "w") as f:
        f.write("\n".join(f"{h}\n{s}" for h, s in entries))

    # Split into chunks
    total = len(entries)
    chunk_size = (total + num_parts - 1) // num_parts
    for i in range(num_parts):
        chunk = entries[i*chunk_size : (i+1)*chunk_size]
        with open(chunk_dir / f"part_{i+1}.fasta", "w") as f:
            f.write("\n".join(f"{h}\n{s}" for h, s in chunk))
    print(f"Wrote {total} entries to {full_fasta_path} and {num_parts} chunks in {chunk_dir}")

dataframe_to_fasta_and_split(bc_df_combined, fasta_dir,id_col="Design",seq_col="Sequence",num_parts=2)


Wrote 10 entries to /data1/lareauc/users/chuh/softwares/CARPNN/examples/01_bindcraft_boltz_refolded/input_tables/monomer_fasta/binder_monomer.fasta and 2 chunks in /data1/lareauc/users/chuh/softwares/CARPNN/examples/01_bindcraft_boltz_refolded/input_tables/monomer_fasta/fasta_chunked


### 3b.2 Run the monomer refolding script

See the README in the AlphaFold workflow for more details on how to set it up

Run the Single sequence mode AlphaFold script `sbatch run_alphafold_monomer_array.sh <input fasta directory> <output directory>`
where `<input fasta directory>` is a directory containing a list of .fasta files. One job will be launched per fasta file.

Example commands:
- `sbatch workflows/AlphaFold/run_alphafold_monomer_array.sh examples/01_bindcraft_boltz_refolded/input_tables/monomer_fasta/fasta_chunked examples/01_bindcraft_boltz_refolded/predictions/af2_monomers`

### 3b.3 Once the monomer folding is done, make a new dataframe that contains both the monomer pdb path and the complex pdb path 

#### 3b.3.1 Load the monomer refolding outputs files


In [14]:
## Replace the directory that contains the monomer outputs with your own
af2_monomer_parental_dir = "/data1/lareauc/users/chuh/softwares/CARPNN/examples/01_bindcraft_boltz_refolded/predictions/af2_monomers"

def load_af2_monomer_df(path):
    af2_monomer_dir = os.path.dirname(path)
    af2_monomer_df = pd.read_csv(path)
    af2_monomer_df["monomer_pdb"] = af2_monomer_df["Output_PDB"].apply(lambda x: os.path.join(af2_monomer_dir, x) + ".pdb")
    return af2_monomer_df

## Load in the monomer files
all_af2_monomer_files = glob.glob(os.path.join(af2_monomer_parental_dir, "*part*", "AF2_score.csv"))
af2_monomer_df = pd.concat([load_af2_monomer_df(f) for f in all_af2_monomer_files], ignore_index=True)
af2_monomer_df


,ID,Name,Sequence,Model/Tag,Output_PDB,lDDT,Time,monomer_pdb
0,0,BCMA_l54_s398596_mpnn9,MKISEGEAHKRFMEIKEEFRRKRKEAEPLGEEEIEKLREEALKEVK...,model_4,BCMA_l54_s398596_mpnn9_model_4.0_r10_af2,93.341,211.3,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
1,1,BCMA_l54_s398596_mpnn17,MKISEGEAHQKFMEIKEEFRRKRKEAEPLGEEEIEKLREEALEKVK...,model_4,BCMA_l54_s398596_mpnn17_model_4.0_r10_af2,92.224,2.9,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
2,2,BCMA_l55_s835555_mpnn4,SMLEKTKEELKKQIEMGEKETVYRVVSYLVEHPERSMMSPEEIEEI...,model_4,BCMA_l55_s835555_mpnn4_model_4.0_r10_af2,95.473,219.2,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
3,3,BCMA_l57_s933667_mpnn1,SWREMSPREYVLEVGVERLREIIREHGGRDSPRFQEFERQFRHWYG...,model_4,BCMA_l57_s933667_mpnn1_model_4.0_r10_af2,90.935,218.6,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
4,4,BCMA_l61_s64905_mpnn4,SEKEMEELKKRFEEVREEARRVWEEKERHREDEEFVEELERRMEEL...,model_4,BCMA_l61_s64905_mpnn4_model_4.0_r10_af2,95.162,218.3,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
5,0,BCMA_l54_s17866_mpnn3,SISDMSVEELEREVEAWEQIARWTEENGMPQWTVERAREMVELYRK...,model_4,BCMA_l54_s17866_mpnn3_model_4.0_r10_af2,95.760,184.1,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
6,1,BCMA_l54_s662851_mpnn3,SMEEYQKKMSEFANEITEEAMKDPEKYKEVSWMLNTEEGRYKLKEI...,model_4,BCMA_l54_s662851_mpnn3_model_4.0_r10_af2,91.802,2.7,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
7,2,BCMA_l58_s790689_mpnn2,SSIEKEIQEEIDIFKYLLSQGMDPEHIKEGIIRRARNEPWFSEEEI...,model_4,BCMA_l58_s790689_mpnn2_model_4.0_r10_af2,93.250,181.6,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
8,3,BCMA_l64_s257143_mpnn2,KLSKEEQRKWIEENKEKVLKVVLSSEHKEWYAGMSWEELKKIFDYV...,model_4,BCMA_l64_s257143_mpnn2_model_4.0_r10_af2,89.909,162.9,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
9,4,BCMA_l64_s257143_mpnn10,MLSKEEQRKWIEENKEKVMKVVLSSEHAEWYAGMSWEELKKIFDYV...,model_4,BCMA_l64_s257143_mpnn10_model_4.0_r10_af2,90.647,2.9,/data1/lareauc/users/chuh/softwares/CARPNN/exa...


#### 3b.3.2 Make a merged dataframe that contains the path to both the monomer pdb and the complex pdb

In [15]:
combined_boltz_df["binder_name"] = combined_boltz_df["prediction_name"].apply(lambda x: x.split("__")[0])
combined_boltz_df_subset = combined_boltz_df[["binder_name","pdb_model_name","pdb_path"]].rename(columns={"pdb_path":"binder_path"})
af2_monomer_df_merged = af2_monomer_df.merge(combined_boltz_df_subset,left_on="Name",right_on="binder_name",how="left")
af2_monomer_df_merged_subset = af2_monomer_df_merged[["binder_name","pdb_model_name","binder_path","monomer_pdb"]]
af2_monomer_df_merged_subset

,binder_name,pdb_model_name,binder_path,monomer_pdb
0,BCMA_l54_s398596_mpnn9,BCMA_l54_s398596_mpnn9__BCMA_model_0,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
1,BCMA_l54_s398596_mpnn9,BCMA_l54_s398596_mpnn9__BCMA_model_4,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
2,BCMA_l54_s398596_mpnn9,BCMA_l54_s398596_mpnn9__BCMA_model_2,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
3,BCMA_l54_s398596_mpnn9,BCMA_l54_s398596_mpnn9__BCMA_model_3,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
4,BCMA_l54_s398596_mpnn9,BCMA_l54_s398596_mpnn9__BCMA_model_1,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
5,BCMA_l54_s398596_mpnn17,BCMA_l54_s398596_mpnn17__BCMA_model_0,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
6,BCMA_l54_s398596_mpnn17,BCMA_l54_s398596_mpnn17__BCMA_model_2,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
7,BCMA_l54_s398596_mpnn17,BCMA_l54_s398596_mpnn17__BCMA_model_4,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
8,BCMA_l54_s398596_mpnn17,BCMA_l54_s398596_mpnn17__BCMA_model_1,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...
9,BCMA_l54_s398596_mpnn17,BCMA_l54_s398596_mpnn17__BCMA_model_3,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...


In [16]:
## Replace the output path name with your own, this table will be used as input for the next command
rmsd_table_path = "/data1/lareauc/users/chuh/softwares/CARPNN/examples/01_bindcraft_boltz_refolded/input_tables/rmsd_table.csv"
af2_monomer_df_merged_subset.to_csv(rmsd_table_path,index=False)

#### 3b.3.3 Run the RMSD script to compute the RMSD between the monomer pdb vs. binder in complex with target pdb
Run `sbatch 09_run_rmsd <path to rmsd_table> <output path>`

Example:
`sbatch workflows/Boltz/09_run_rmsd.sh examples/01_bindcraft_boltz_refolded/input_tables/rmsd_table.csv examples/01_bindcraft_boltz_refolded/input_tables/rmsd_table_monomer.csv`

#### 3.3.4 After the RMSD calculation finishes running, add monomer_lDDT and monomer_RMSD to the dataframe

In [17]:
## Replace path with the output from the pipeline
rmsd_monomer_path = "/data1/lareauc/users/chuh/softwares/CARPNN/examples/01_bindcraft_boltz_refolded/input_tables/rmsd_table_monomer.csv"

rmsd_monomer = pd.read_csv(rmsd_monomer_path)
rmsd_monomer = rmsd_monomer.rename(columns={"binder_complex_monomer_rmsd":"binder_monomer_RMSD"})
rmsd_merged = rmsd_monomer.merge(af2_monomer_df,on="monomer_pdb")
rmsd_merged_subset = rmsd_merged[["pdb_model_name","lDDT","binder_monomer_RMSD"]].rename(columns={"lDDT":"monomer_lDDT"})
rmsd_merged_subset

,pdb_model_name,monomer_lDDT,binder_monomer_RMSD
0,BCMA_l54_s398596_mpnn9__BCMA_model_0,93.341,0.509
1,BCMA_l54_s398596_mpnn9__BCMA_model_4,93.341,0.511
2,BCMA_l54_s398596_mpnn9__BCMA_model_2,93.341,0.508
3,BCMA_l54_s398596_mpnn9__BCMA_model_3,93.341,0.484
4,BCMA_l54_s398596_mpnn9__BCMA_model_1,93.341,0.442
5,BCMA_l54_s398596_mpnn17__BCMA_model_0,92.224,0.527
6,BCMA_l54_s398596_mpnn17__BCMA_model_2,92.224,0.365
7,BCMA_l54_s398596_mpnn17__BCMA_model_4,92.224,0.548
8,BCMA_l54_s398596_mpnn17__BCMA_model_1,92.224,0.447
9,BCMA_l54_s398596_mpnn17__BCMA_model_3,92.224,0.402


In [18]:
## Merge the information back to the dataframe
combined_boltz_df_rmsd_merged = combined_boltz_df.merge(rmsd_merged_subset,on="pdb_model_name",how="left")
combined_boltz_df_rmsd_merged

,prediction_name,pdb_model_name,pdb_path,confidence_path,pae_path,plddt_path,confidence_score,ptm,iptm,ligand_iptm,protein_iptm,complex_plddt,complex_iplddt,complex_pde,complex_ipde,chains_ptm_0,chains_ptm_1,pair_chains_iptm_0_0,pair_chains_iptm_0_1,pair_chains_iptm_1_0,pair_chains_iptm_1_1,pae_all,pae_chain_a,pae_chain_b,pae_interaction,Chn1,Chn2,PAE,Dist,Type,ipSAE,ipSAE_d0chn,ipSAE_d0dom,ipTM_af,ipTM_d0chn,pDockQ,pDockQ2,LIS,n0res,n0chn,n0dom,d0res,d0chn,d0dom,nres1,nres2,dist1,dist2,Model,i_pLDDT,ss_pLDDT,Unrelaxed_Clashes,Relaxed_Clashes,Binder_Energy_Score,Surface_Hydrophobicity,ShapeComplementarity,PackStat,dG,dSASA,dG/dSASA,Interface_SASA_%,Interface_Hydrophobicity,n_InterfaceResidues,n_InterfaceHbonds,InterfaceHbondsPercentage,n_InterfaceUnsatHbonds,InterfaceUnsatHbondsPercentage,InterfaceAAs,Interface_Helix%,Interface_BetaSheet%,Interface_Loop%,Binder_Helix%,Binder_BetaSheet%,Binder_Loop%,binder_path,binder_relaxed_path,id,chain_a_seq,chain_b_seq,binder_name,monomer_lDDT,binder_monomer_RMSD
0,BCMA_l61_s64905_mpnn4__BCMA,BCMA_l61_s64905_mpnn4__BCMA_model_2,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.936145,0.955550,0.945506,0.0,0.945506,0.933805,0.941414,0.278135,0.340065,0.965639,0.965048,0.965639,0.923435,0.945506,0.965048,1.522581,1.169044,1.158353,1.929582,A,B,10,10,max,0.883050,0.950799,0.950799,0.946,0.950799,0.3669,0.8799,0.8408,61,97,97,2.64,3.59,3.59,61,36,26,16,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.96,0.97,0,0,-170.26,0.20,0.67,0.68,-41.56,1425.31,-2.92,31.27,66.67,18,6,33.33,1.0,5.56,"{'A': 1, 'C': 0, 'D': 0, 'E': 1, 'F': 1, 'G': ...",100.00,0.0,0.00,90.16,0.0,9.84,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,BCMA_l61_s64905_mpnn4__BCMA,SEKEMEELKKRFEEVREEARRVWEEKERHREDEEFVEELERRMEEL...,CSQNEYFDSLLHACIPCQLRCSSNTPPLTCQRYCNA,BCMA_l61_s64905_mpnn4,95.162,0.359
1,BCMA_l61_s64905_mpnn4__BCMA,BCMA_l61_s64905_mpnn4__BCMA_model_0,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.937535,0.951433,0.939903,0.0,0.939903,0.936943,0.947970,0.288089,0.367192,0.963098,0.962673,0.963098,0.923888,0.939903,0.962673,1.662470,1.304377,1.240707,2.090309,A,B,10,10,max,0.881840,0.943222,0.943222,0.940,0.943222,0.3881,0.8957,0.8284,61,97,97,2.64,3.59,3.59,61,36,28,17,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.97,0.97,2,0,-168.24,0.20,0.72,0.64,-41.22,1408.31,-2.93,31.00,66.67,18,6,33.33,1.0,5.56,"{'A': 1, 'C': 0, 'D': 0, 'E': 1, 'F': 1, 'G': ...",100.00,0.0,0.00,90.16,0.0,9.84,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,BCMA_l61_s64905_mpnn4__BCMA,SEKEMEELKKRFEEVREEARRVWEEKERHREDEEFVEELERRMEEL...,CSQNEYFDSLLHACIPCQLRCSSNTPPLTCQRYCNA,BCMA_l61_s64905_mpnn4,95.162,0.369
2,BCMA_l61_s64905_mpnn4__BCMA,BCMA_l61_s64905_mpnn4__BCMA_model_3,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.935590,0.947600,0.933816,0.0,0.933816,0.936033,0.948799,0.296900,0.399565,0.962050,0.957838,0.962050,0.923114,0.933816,0.957838,1.754216,1.332538,1.355371,2.229163,A,B,10,10,max,0.878208,0.934759,0.934759,0.934,0.934759,0.3403,0.8992,0.8219,61,97,97,2.64,3.59,3.59,61,36,28,16,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,0.97,0.97,1,0,-176.05,0.23,0.73,0.65,-38.47,1371.93,-2.80,30.77,66.67,18,3,16.67,0.0,0.00,"{'A': 1, 'C': 0, 'D': 0, 'E': 1, 'F': 1, 'G': ...",100.00,0.0,0.00,90.16,0.0,9.84,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,/data1/lareauc/users/chuh/softwares/CARPNN/exa...,BCMA_l61_s64905_mpnn4__BCMA,SEKEMEELKKRFEEVREEARRVWEEKERHREDEEFVEELERRMEEL...,CS

### 3.4 Transform the long-form prediction outputs into a BindCraft style wide-form format
These functions will convert the table from one prediction per row to one sequence per row where each sequence now has the additional columns for each prediciton

In [21]:
def sort_columns_by_feature_group(col_list):
    '''
    This is a helper function that sorts the column numerically
    '''
    grouped = defaultdict(list)
    for col in col_list:
        parts = col.split("_", 1)
        rest = parts[1] if len(parts) > 1 else parts[0]
        match = re.match(r"([a-zA-Z0-9_%]+)", rest)
        base = match.group(1) if match else rest
        grouped[base].append(col)

    def numeric_suffix_sort_key(name):
        nums = re.findall(r'\d+', name)
        return [int(n) for n in nums] if nums else [float('inf')]

    sorted_col_groups = {}
    for base in sorted(grouped.keys()):
        sorted_group = sorted(grouped[base], key=numeric_suffix_sort_key)
        sorted_col_groups[base] = sorted_group
    return sorted_col_groups

def average_dicts_across_columns(df, cols):
    '''
    This is a helper function that computes the 'Average' column across prediction columns
    '''
    all_keys = set()

    def safe_dict(d):
        if isinstance(d, dict):
            return d
        try:
            return ast.literal_eval(d)
        except:
            return {}

    for col in cols:
        all_keys.update(*df[col].dropna().apply(lambda d: safe_dict(d).keys()))

    def average_dicts(row):
        agg = {k: [] for k in all_keys}
        for col in cols:
            d = safe_dict(row[col])
            for k, v in d.items():
                agg[k].append(v)
        return {k: float(np.mean(v)) if v else 0.0 for k, v in agg.items()}

    return df[cols].apply(average_dicts, axis=1)

def boltz_to_bindcraft_df(df):
    '''
    This is the main function that converts the prediction table into a BindCraft-like format
    Adjust the columns included for naming updates / missing information if needed.
    For example: If you did not run the monomer refolding, omit the rmsd_columns
    '''
    boltz_columns = [
        "confidence_score","ptm","iptm","ligand_iptm","protein_iptm",
        "complex_plddt","complex_iplddt","complex_pde","complex_ipde",
        "chains_ptm_0","chains_ptm_1",
        "pair_chains_iptm_0_0","pair_chains_iptm_0_1","pair_chains_iptm_1_0","pair_chains_iptm_1_1"
    ]
    bindcraft_columns = [
        "i_pLDDT","ss_pLDDT","Unrelaxed_Clashes","Relaxed_Clashes","Binder_Energy_Score",
        "Surface_Hydrophobicity","ShapeComplementarity","PackStat","dG","dSASA","dG/dSASA",
        "Interface_SASA_%","Interface_Hydrophobicity","n_InterfaceResidues","n_InterfaceHbonds",
        "InterfaceHbondsPercentage","n_InterfaceUnsatHbonds","InterfaceUnsatHbondsPercentage",
        "InterfaceAAs","Interface_Helix%","Interface_BetaSheet%","Interface_Loop%","Binder_Helix%",
        "Binder_BetaSheet%","Binder_Loop%"
    ]
    rmsd_columns = [
        "binder_monomer_RMSD","monomer_lDDT"
    ]
    ipsae_columns = [
        "ipSAE","ipSAE_d0chn","ipSAE_d0dom","ipTM_af","ipTM_d0chn","pDockQ","pDockQ2","LIS"
    ]
    pae_columns = [
        "pae_all","pae_chain_a","pae_chain_b","pae_interaction"
    ]
    ## If you want normalized pAE, you can also divide the pAE columns by 31.5
    ## Fun fact: BindCraft ipAE was normalized by 31
    #df[pae_columns] = df[pae_columns]/31.5
    columns_to_melt = boltz_columns + ipsae_columns + pae_columns + rmsd_columns + bindcraft_columns 

    df["Model"] = df["pdb_model_name"].apply(lambda x: int(x.split("_model_")[1][0]))
    ## For human readability we increment the model numbers by 1
    df["Model_str"] = df["Model"].apply(lambda x: str(x+1))

    df_long = df.melt(id_vars=['prediction_name', 'Model_str'], value_vars=columns_to_melt,
                      var_name='measurement', value_name='value')

    df_wide = df_long.pivot(index='prediction_name', columns=['Model_str', "measurement"], values='value')
    df_wide.columns = ['_'.join(col).rstrip("_") for col in df_wide.columns]
    sorted_col_groups = sort_columns_by_feature_group(df_wide.columns)
    blocks = []
    for base, cols in sorted_col_groups.items():
        block = df_wide[cols].copy()
        if base == "InterfaceAAs":
            avg_series = average_dicts_across_columns(block, cols)
        else:
            avg_series = block.mean(axis=1)

        block.insert(0, f"Average_{base.rstrip('_')}", avg_series)
        blocks.append(block)
    concat_df = pd.concat(blocks, axis=1).reset_index()
    return concat_df


In [22]:
## Convert the long format merged dataframe into boltz wide format
boltz_bindcraft_style_df = boltz_to_bindcraft_df(combined_boltz_df_rmsd_merged)

## If the monomer columns are not given, remove the RMSD columns in the boltz_to_bindcraft_df function and run this line instead
# boltz_bindcraft_style_df = boltz_to_bindcraft_df(combined_boltz_df)

boltz_bindcraft_style_df


,prediction_name,Average_Binder_BetaSheet%,1_Binder_BetaSheet%,2_Binder_BetaSheet%,3_Binder_BetaSheet%,4_Binder_BetaSheet%,5_Binder_BetaSheet%,Average_Binder_Energy_Score,1_Binder_Energy_Score,2_Binder_Energy_Score,3_Binder_Energy_Score,4_Binder_Energy_Score,5_Binder_Energy_Score,Average_Binder_Helix%,1_Binder_Helix%,2_Binder_Helix%,3_Binder_Helix%,4_Binder_Helix%,5_Binder_Helix%,Average_Binder_Loop%,1_Binder_Loop%,2_Binder_Loop%,3_Binder_Loop%,4_Binder_Loop%,5_Binder_Loop%,Average_InterfaceAAs,1_InterfaceAAs,2_InterfaceAAs,3_InterfaceAAs,4_InterfaceAAs,5_InterfaceAAs,Average_InterfaceHbondsPercentage,1_InterfaceHbondsPercentage,2_InterfaceHbondsPercentage,3_InterfaceHbondsPercentage,4_InterfaceHbondsPercentage,5_InterfaceHbondsPercentage,Average_InterfaceUnsatHbondsPercentage,1_InterfaceUnsatHbondsPercentage,2_InterfaceUnsatHbondsPercentage,3_InterfaceUnsatHbondsPercentage,4_InterfaceUnsatHbondsPercentage,5_InterfaceUnsatHbondsPercentage,Average_Interface_BetaSheet%,1_Interface_BetaSheet%,2_Interface_BetaSheet%,3_Interface_BetaSheet%,4_Interface_BetaSheet%,5_Interface_BetaSheet%,Average_Interface_Helix%,1_Interface_Helix%,2_Interface_Helix%,3_Interface_Helix%,4_Interface_Helix%,5_Interface_Helix%,Average_Interface_Hydrophobicity,1_Interface_Hydrophobicity,2_Interface_Hydrophobicity,3_Interface_Hydrophobicity,4_Interface_Hydrophobicity,5_Interface_Hydrophobicity,Average_Interface_Loop%,1_Interface_Loop%,2_Interface_Loop%,3_Interface_Loop%,4_Interface_Loop%,5_Interface_Loop%,Average_Interface_SASA_%,1_Interface_SASA_%,2_Interface_SASA_%,3_Interface_SASA_%,4_Interface_SASA_%,5_Interface_SASA_%,Average_LIS,1_LIS,2_LIS,3_LIS,4_LIS,5_LIS,Average_PackStat,1_PackStat,2_PackStat,3_PackStat,4_PackStat,5_PackStat,Average_Relaxed_Clashes,1_Relaxed_Clashes,2_Relaxed_Clashes,3_Relaxed_Clashes,4_Relaxed_Clashes,5_Relaxed_Clashes,Average_ShapeComplementarity,1_ShapeComplementarity,2_ShapeComplementarity,3_ShapeComplementarity,4_ShapeComplementarity,5_ShapeComplementarity,Average_Surface_Hydrophobicity,1_Surface_Hydrophobicity,2_Surface_Hydrophobicity,3_Surface_Hydrophobicity,4_Surface_Hydrophobicity,5_Surface_Hydrophobicity,Average_Unrelaxed_Clashes,1_Unrelaxed_Clashes,2_Unrelaxed_Clashes,3_Unrelaxed_Clashes,4_Unrelaxed_Clashes,5_Unrelaxed_Clashes,Average_binder_monomer_RMSD,1_binder_monomer_RMSD,2_binder_monomer_RMSD,3_binder_monomer_RMSD,4_binder_monomer_RMSD,5_binder_monomer_RMSD,Average_chains_ptm_0,1_chains_ptm_0,2_chains_ptm_0,3_chains_ptm_0,4_chains_ptm_0,5_chains_ptm_0,Average_chains_ptm_1,1_chains_ptm_1,2_chains_ptm_1,3_chains_ptm_1,4_chains_ptm_1,5_chains_ptm_1,Average_complex_ipde,1_complex_ipde,2_complex_ipde,3_complex_ipde,4_complex_ipde,5_complex_ipde,Average_complex_iplddt,1_complex_iplddt,2_complex_iplddt,3_complex_iplddt,4_complex_iplddt,5_complex_iplddt,Average_complex_pde,1_complex_pde,2_complex_pde,3_complex_pde,4_complex_pde,5_complex_pde,Average_complex_plddt,1_complex_plddt,2_complex_plddt,3_complex_plddt,4_complex_plddt,5_complex_plddt,Average_confidence_score,1_confidence_score,2_confidence_score,3_confidence_score,4_confidence_score,5_confidence_score,Average_dG,1_dG,1_dG/dSASA,2_dG,2_dG/dSASA,3_dG,3_dG/dSASA,4_dG,4_dG/dSASA,5_dG,5_dG/dSASA,Average_dSASA,1_dSASA,2_dSASA,3_dSASA,4_dSASA,5_dSASA,Average_i_pLDDT,1_i_pLDDT,2_i_pLDDT,3_i_pLDDT,4_i_pLDDT,5_i_pLDDT,Average_ipSAE,1_ipSAE,2_ipSAE,3_ipSAE,4_ipSAE,5_ipSAE,Average_ipSAE_d0chn,1_ipSAE_d0chn,2_ipSAE_d0chn,3_ipSAE_d0chn,4_ipSAE_d0chn,5_ipSAE_d0chn,Average_ipSAE_d0dom,1_ipSAE_d0dom,2_ipSAE_d0dom,3_ipSAE_d0dom,4_ipSAE_d0dom,5_ipSAE_d0dom,Average_ipTM_af,1_ipTM_af,2_ipTM_af,3_ipTM_af,4_ipTM_af,5_ipTM_af,Average_ipTM_d0chn,1_ipTM_d0chn,2_ipTM_d0chn,3_ipTM_d0chn,4_ipTM_d0chn,5_ipTM_d0chn,Average_iptm,1_iptm,2_iptm,3_iptm,4_iptm,5_iptm,Average_ligand_iptm,1_ligand_iptm,2_ligand_iptm,3_ligand_iptm,4_ligand_iptm,5_ligand_iptm,Average_monomer_lDDT,1_monomer_lDDT,2_monomer_lDDT,3_monomer_lDDT,4_monomer_lDDT,5_monomer_lDDT,Average_n_InterfaceHbonds,1_n_InterfaceHbonds,2_n_In

In [23]:
## Add sequence information
boltz_bindcraft_style_df["binder_name"] = boltz_bindcraft_style_df["prediction_name"].apply(lambda x: x.split("__")[0])
boltz_bindcraft_style_df = boltz_bindcraft_style_df.merge(combined_boltz_df[["binder_name","chain_a_seq"]].drop_duplicates(),on="binder_name")
boltz_bindcraft_style_df

,prediction_name,Average_Binder_BetaSheet%,1_Binder_BetaSheet%,2_Binder_BetaSheet%,3_Binder_BetaSheet%,4_Binder_BetaSheet%,5_Binder_BetaSheet%,Average_Binder_Energy_Score,1_Binder_Energy_Score,2_Binder_Energy_Score,3_Binder_Energy_Score,4_Binder_Energy_Score,5_Binder_Energy_Score,Average_Binder_Helix%,1_Binder_Helix%,2_Binder_Helix%,3_Binder_Helix%,4_Binder_Helix%,5_Binder_Helix%,Average_Binder_Loop%,1_Binder_Loop%,2_Binder_Loop%,3_Binder_Loop%,4_Binder_Loop%,5_Binder_Loop%,Average_InterfaceAAs,1_InterfaceAAs,2_InterfaceAAs,3_InterfaceAAs,4_InterfaceAAs,5_InterfaceAAs,Average_InterfaceHbondsPercentage,1_InterfaceHbondsPercentage,2_InterfaceHbondsPercentage,3_InterfaceHbondsPercentage,4_InterfaceHbondsPercentage,5_InterfaceHbondsPercentage,Average_InterfaceUnsatHbondsPercentage,1_InterfaceUnsatHbondsPercentage,2_InterfaceUnsatHbondsPercentage,3_InterfaceUnsatHbondsPercentage,4_InterfaceUnsatHbondsPercentage,5_InterfaceUnsatHbondsPercentage,Average_Interface_BetaSheet%,1_Interface_BetaSheet%,2_Interface_BetaSheet%,3_Interface_BetaSheet%,4_Interface_BetaSheet%,5_Interface_BetaSheet%,Average_Interface_Helix%,1_Interface_Helix%,2_Interface_Helix%,3_Interface_Helix%,4_Interface_Helix%,5_Interface_Helix%,Average_Interface_Hydrophobicity,1_Interface_Hydrophobicity,2_Interface_Hydrophobicity,3_Interface_Hydrophobicity,4_Interface_Hydrophobicity,5_Interface_Hydrophobicity,Average_Interface_Loop%,1_Interface_Loop%,2_Interface_Loop%,3_Interface_Loop%,4_Interface_Loop%,5_Interface_Loop%,Average_Interface_SASA_%,1_Interface_SASA_%,2_Interface_SASA_%,3_Interface_SASA_%,4_Interface_SASA_%,5_Interface_SASA_%,Average_LIS,1_LIS,2_LIS,3_LIS,4_LIS,5_LIS,Average_PackStat,1_PackStat,2_PackStat,3_PackStat,4_PackStat,5_PackStat,Average_Relaxed_Clashes,1_Relaxed_Clashes,2_Relaxed_Clashes,3_Relaxed_Clashes,4_Relaxed_Clashes,5_Relaxed_Clashes,Average_ShapeComplementarity,1_ShapeComplementarity,2_ShapeComplementarity,3_ShapeComplementarity,4_ShapeComplementarity,5_ShapeComplementarity,Average_Surface_Hydrophobicity,1_Surface_Hydrophobicity,2_Surface_Hydrophobicity,3_Surface_Hydrophobicity,4_Surface_Hydrophobicity,5_Surface_Hydrophobicity,Average_Unrelaxed_Clashes,1_Unrelaxed_Clashes,2_Unrelaxed_Clashes,3_Unrelaxed_Clashes,4_Unrelaxed_Clashes,5_Unrelaxed_Clashes,Average_binder_monomer_RMSD,1_binder_monomer_RMSD,2_binder_monomer_RMSD,3_binder_monomer_RMSD,4_binder_monomer_RMSD,5_binder_monomer_RMSD,Average_chains_ptm_0,1_chains_ptm_0,2_chains_ptm_0,3_chains_ptm_0,4_chains_ptm_0,5_chains_ptm_0,Average_chains_ptm_1,1_chains_ptm_1,2_chains_ptm_1,3_chains_ptm_1,4_chains_ptm_1,5_chains_ptm_1,Average_complex_ipde,1_complex_ipde,2_complex_ipde,3_complex_ipde,4_complex_ipde,5_complex_ipde,Average_complex_iplddt,1_complex_iplddt,2_complex_iplddt,3_complex_iplddt,4_complex_iplddt,5_complex_iplddt,Average_complex_pde,1_complex_pde,2_complex_pde,3_complex_pde,4_complex_pde,5_complex_pde,Average_complex_plddt,1_complex_plddt,2_complex_plddt,3_complex_plddt,4_complex_plddt,5_complex_plddt,Average_confidence_score,1_confidence_score,2_confidence_score,3_confidence_score,4_confidence_score,5_confidence_score,Average_dG,1_dG,1_dG/dSASA,2_dG,2_dG/dSASA,3_dG,3_dG/dSASA,4_dG,4_dG/dSASA,5_dG,5_dG/dSASA,Average_dSASA,1_dSASA,2_dSASA,3_dSASA,4_dSASA,5_dSASA,Average_i_pLDDT,1_i_pLDDT,2_i_pLDDT,3_i_pLDDT,4_i_pLDDT,5_i_pLDDT,Average_ipSAE,1_ipSAE,2_ipSAE,3_ipSAE,4_ipSAE,5_ipSAE,Average_ipSAE_d0chn,1_ipSAE_d0chn,2_ipSAE_d0chn,3_ipSAE_d0chn,4_ipSAE_d0chn,5_ipSAE_d0chn,Average_ipSAE_d0dom,1_ipSAE_d0dom,2_ipSAE_d0dom,3_ipSAE_d0dom,4_ipSAE_d0dom,5_ipSAE_d0dom,Average_ipTM_af,1_ipTM_af,2_ipTM_af,3_ipTM_af,4_ipTM_af,5_ipTM_af,Average_ipTM_d0chn,1_ipTM_d0chn,2_ipTM_d0chn,3_ipTM_d0chn,4_ipTM_d0chn,5_ipTM_d0chn,Average_iptm,1_iptm,2_iptm,3_iptm,4_iptm,5_iptm,Average_ligand_iptm,1_ligand_iptm,2_ligand_iptm,3_ligand_iptm,4_ligand_iptm,5_ligand_iptm,Average_monomer_lDDT,1_monomer_lDDT,2_monomer_lDDT,3_monomer_lDDT,4_monomer_lDDT,5_monomer_lDDT,Average_n_InterfaceHbonds,1_n_InterfaceHbonds,2_n_In

## Step 4: Apply BindCraft filter on Boltz outputs

In [24]:
## Makes sure the filter applied is correct
def apply_bindcraft_filter(df, json_filters_path=None, json_filters=None):
    '''
    This is a helper function that reads in the json and loops through every row to check for unmet conditions
    '''
    if json_filters is None:
        with open(json_filters_path, "r") as file:
            json_filters = json.load(file)
    unmet_conditions = []
    for idx, row in tqdm.tqdm(df.iterrows(), total=len(df)):
        unmet_conditions.append(check_row_pass_filter(row, json_filters))
    df["unmet_conditions"] = unmet_conditions
    return df

## Makes sure the filter applied is correct
def check_row_pass_filter(row, filter_json):
    '''
    This function checks if the given row fulfills the conditions specified in the filter json
    
    Note that if a column in the filter is not present in the dataframe that filter will be skipped so
    make sure all the columns are present if you are making a custom filter
    '''
    unmet_conditions = []
    # Check filters against thresholds
    for label, conditions in filter_json.items():
        # Special conditions for interface amino acid counts
        if label.startswith("Average_InterfaceAAs") or any(label.startswith(f"{i}_InterfaceAAs") for i in range(1, 6)):
            for aa, aa_conditions in conditions.items():
                row_label = row.get(label)
                if row_label is None or row_label != row_label:  # Check for NaN
                    continue
                if isinstance(row_label, str):
                    try:
                        json_value = ast.literal_eval(row_label)
                    except Exception as e:
                        continue  # or log the error
                elif isinstance(row_label, dict):
                    json_value = row_label
                else:
                    continue
                value = json_value.get(aa)
                if value is None or aa_conditions["threshold"] is None:
                    continue
                if aa_conditions["higher"]:
                    if value < aa_conditions["threshold"]:
                        unmet_conditions.append(f"{label}_{aa}")
                else:
                    if value > aa_conditions["threshold"]:
                        unmet_conditions.append(f"{label}_{aa}")
        else:
            # Generic condition check
            value = row.get(label)
            if value is None or conditions["threshold"] is None:
                continue
            if conditions["higher"]:
                if value < conditions["threshold"]:
                    unmet_conditions.append(label)
            else:
                if value > conditions["threshold"]:
                    unmet_conditions.append(label)
    return unmet_conditions


### 4.1 Pick a filter file to use
We used a Boltz filter that closely resembles BindCraft's filter conditions and provide a few variations:
- `Boltz1_default_filters` The default filter
- `Boltz1_unrelaxed_filters` If the predicted structures are unrelaxed the dG could be higher than usual so we relax the dG requirement
- `Boltz1_default_filters_Lowered_UnsatHBonds` In some campaigns we found the number of Unsaturated Hbond requirement is too strict so we relax it as well

These filters are made based on empirical results and have not been throughly tested. Adjust them to your needs as you see fit!

In [29]:
boltz1_binder_filter_path = "/data1/lareauc/users/chuh/softwares/CARPNN/filters/Boltz1_default_filters.json"
#boltz1_binder_filter_path = "/data1/lareauc/users/chuh/softwares/CARPNN/filters/Boltz1_unrelaxed_filters.json"
#boltz1_binder_filter_path = "/data1/lareauc/users/chuh/softwares/CARPNN/filters/Boltz1_default_filters_Lowered_UnsatHBonds.json"

boltz_bindcraft_style_df = apply_bindcraft_filter(boltz_bindcraft_style_df,json_filters_path=boltz1_binder_filter_path)
boltz_bindcraft_style_df["unmet_conditions_length"] = boltz_bindcraft_style_df["unmet_conditions"].apply(len)
boltz_bindcraft_style_df


100%|██████████| 10/10 [00:00<00:00, 1100.03it/s]


,prediction_name,Average_Binder_BetaSheet%,1_Binder_BetaSheet%,2_Binder_BetaSheet%,3_Binder_BetaSheet%,4_Binder_BetaSheet%,5_Binder_BetaSheet%,Average_Binder_Energy_Score,1_Binder_Energy_Score,2_Binder_Energy_Score,3_Binder_Energy_Score,4_Binder_Energy_Score,5_Binder_Energy_Score,Average_Binder_Helix%,1_Binder_Helix%,2_Binder_Helix%,3_Binder_Helix%,4_Binder_Helix%,5_Binder_Helix%,Average_Binder_Loop%,1_Binder_Loop%,2_Binder_Loop%,3_Binder_Loop%,4_Binder_Loop%,5_Binder_Loop%,Average_InterfaceAAs,1_InterfaceAAs,2_InterfaceAAs,3_InterfaceAAs,4_InterfaceAAs,5_InterfaceAAs,Average_InterfaceHbondsPercentage,1_InterfaceHbondsPercentage,2_InterfaceHbondsPercentage,3_InterfaceHbondsPercentage,4_InterfaceHbondsPercentage,5_InterfaceHbondsPercentage,Average_InterfaceUnsatHbondsPercentage,1_InterfaceUnsatHbondsPercentage,2_InterfaceUnsatHbondsPercentage,3_InterfaceUnsatHbondsPercentage,4_InterfaceUnsatHbondsPercentage,5_InterfaceUnsatHbondsPercentage,Average_Interface_BetaSheet%,1_Interface_BetaSheet%,2_Interface_BetaSheet%,3_Interface_BetaSheet%,4_Interface_BetaSheet%,5_Interface_BetaSheet%,Average_Interface_Helix%,1_Interface_Helix%,2_Interface_Helix%,3_Interface_Helix%,4_Interface_Helix%,5_Interface_Helix%,Average_Interface_Hydrophobicity,1_Interface_Hydrophobicity,2_Interface_Hydrophobicity,3_Interface_Hydrophobicity,4_Interface_Hydrophobicity,5_Interface_Hydrophobicity,Average_Interface_Loop%,1_Interface_Loop%,2_Interface_Loop%,3_Interface_Loop%,4_Interface_Loop%,5_Interface_Loop%,Average_Interface_SASA_%,1_Interface_SASA_%,2_Interface_SASA_%,3_Interface_SASA_%,4_Interface_SASA_%,5_Interface_SASA_%,Average_LIS,1_LIS,2_LIS,3_LIS,4_LIS,5_LIS,Average_PackStat,1_PackStat,2_PackStat,3_PackStat,4_PackStat,5_PackStat,Average_Relaxed_Clashes,1_Relaxed_Clashes,2_Relaxed_Clashes,3_Relaxed_Clashes,4_Relaxed_Clashes,5_Relaxed_Clashes,Average_ShapeComplementarity,1_ShapeComplementarity,2_ShapeComplementarity,3_ShapeComplementarity,4_ShapeComplementarity,5_ShapeComplementarity,Average_Surface_Hydrophobicity,1_Surface_Hydrophobicity,2_Surface_Hydrophobicity,3_Surface_Hydrophobicity,4_Surface_Hydrophobicity,5_Surface_Hydrophobicity,Average_Unrelaxed_Clashes,1_Unrelaxed_Clashes,2_Unrelaxed_Clashes,3_Unrelaxed_Clashes,4_Unrelaxed_Clashes,5_Unrelaxed_Clashes,Average_binder_monomer_RMSD,1_binder_monomer_RMSD,2_binder_monomer_RMSD,3_binder_monomer_RMSD,4_binder_monomer_RMSD,5_binder_monomer_RMSD,Average_chains_ptm_0,1_chains_ptm_0,2_chains_ptm_0,3_chains_ptm_0,4_chains_ptm_0,5_chains_ptm_0,Average_chains_ptm_1,1_chains_ptm_1,2_chains_ptm_1,3_chains_ptm_1,4_chains_ptm_1,5_chains_ptm_1,Average_complex_ipde,1_complex_ipde,2_complex_ipde,3_complex_ipde,4_complex_ipde,5_complex_ipde,Average_complex_iplddt,1_complex_iplddt,2_complex_iplddt,3_complex_iplddt,4_complex_iplddt,5_complex_iplddt,Average_complex_pde,1_complex_pde,2_complex_pde,3_complex_pde,4_complex_pde,5_complex_pde,Average_complex_plddt,1_complex_plddt,2_complex_plddt,3_complex_plddt,4_complex_plddt,5_complex_plddt,Average_confidence_score,1_confidence_score,2_confidence_score,3_confidence_score,4_confidence_score,5_confidence_score,Average_dG,1_dG,1_dG/dSASA,2_dG,2_dG/dSASA,3_dG,3_dG/dSASA,4_dG,4_dG/dSASA,5_dG,5_dG/dSASA,Average_dSASA,1_dSASA,2_dSASA,3_dSASA,4_dSASA,5_dSASA,Average_i_pLDDT,1_i_pLDDT,2_i_pLDDT,3_i_pLDDT,4_i_pLDDT,5_i_pLDDT,Average_ipSAE,1_ipSAE,2_ipSAE,3_ipSAE,4_ipSAE,5_ipSAE,Average_ipSAE_d0chn,1_ipSAE_d0chn,2_ipSAE_d0chn,3_ipSAE_d0chn,4_ipSAE_d0chn,5_ipSAE_d0chn,Average_ipSAE_d0dom,1_ipSAE_d0dom,2_ipSAE_d0dom,3_ipSAE_d0dom,4_ipSAE_d0dom,5_ipSAE_d0dom,Average_ipTM_af,1_ipTM_af,2_ipTM_af,3_ipTM_af,4_ipTM_af,5_ipTM_af,Average_ipTM_d0chn,1_ipTM_d0chn,2_ipTM_d0chn,3_ipTM_d0chn,4_ipTM_d0chn,5_ipTM_d0chn,Average_iptm,1_iptm,2_iptm,3_iptm,4_iptm,5_iptm,Average_ligand_iptm,1_ligand_iptm,2_ligand_iptm,3_ligand_iptm,4_ligand_iptm,5_ligand_iptm,Average_monomer_lDDT,1_monomer_lDDT,2_monomer_lDDT,3_monomer_lDDT,4_monomer_lDDT,5_monomer_lDDT,Average_n_InterfaceHbonds,1_n_InterfaceHbonds,2_n_In

### 4.2 Check how many entries passed filters (possible that none has passed)


In [31]:
len(boltz_bindcraft_style_df[boltz_bindcraft_style_df["unmet_conditions_length"]==0])

7

### 4.3 Check which condition failed the most and consider adjusting the filter thresold if no entries passed

In [32]:
## Look at which conditions are the most frequently unmet and adjust accordingly
unmet_items = [item for sublist in boltz_bindcraft_style_df['unmet_conditions'] for item in sublist]
unmet_counter = Counter(unmet_items)
unmet_freq_df = pd.DataFrame(unmet_counter.items(), columns=["item", "count"]).sort_values("count", ascending=False)
unmet_freq_df.columns = ["Filter name","# of Binder Failed filter"]
unmet_freq_df

## Make a copy of the filter and update accordingly

,Filter name,# of Binder Failed filter
1,2_n_InterfaceHbonds,2
0,1_n_InterfaceHbonds,1
2,Average_n_InterfaceUnsatHbonds,1
3,2_n_InterfaceUnsatHbonds,1


### 4.4 Apply other additional post-hoc filters 

In [41]:
## Calculate the net charge of the binder
def net_charge(seq, pH=7.4):
    pKa = {
        'C_term': 3.1, 'N_term': 8.0,
        'D': 3.9, 'E': 4.1, 'C': 8.3, 'Y': 10.1,
        'H': 6.0, 'K': 10.5, 'R': 12.5
    }
    counts = Counter(seq)
    neg = sum(counts[aa] * (1 / (1 + 10**(pKa[aa] - pH))) for aa in ['D', 'E', 'C', 'Y'])
    pos = sum(counts[aa] * (1 / (1 + 10**(pH - pKa[aa]))) for aa in ['H', 'K', 'R'])
    pos += 1 / (1 + 10**(pH - pKa['N_term']))
    neg += 1 / (1 + 10**(pKa['C_term'] - pH))
    return pos - neg

## As we shown in the manuscript we want to avoid binders that are positively charged
boltz_bindcraft_style_df["binder_seq_net_charge"] = boltz_bindcraft_style_df["chain_a_seq"].apply(net_charge)

## We also like ipSAE > 0.85 as a cutoff
boltz_bindcraft_style_df_filtered = boltz_bindcraft_style_df[
(boltz_bindcraft_style_df["unmet_conditions_length"]==0)&
(boltz_bindcraft_style_df["binder_seq_net_charge"]<0)&
(boltz_bindcraft_style_df["Average_ipSAE"]>=0.85)
].sort_values("Average_ipSAE",ascending=False) ## Sorting from high to low by ipSAE is recommended

## (Optional) To enforce diversity in backbone, we only include one mpnn draws per backbone
boltz_bindcraft_style_df_filtered["backbone_name"] = boltz_bindcraft_style_df_filtered["prediction_name"].apply(lambda x: x.split("_mpnn")[0])
boltz_bindcraft_style_df_filtered = boltz_bindcraft_style_df_filtered.drop_duplicates("backbone_name")
boltz_bindcraft_style_df_filtered

,prediction_name,Average_Binder_BetaSheet%,1_Binder_BetaSheet%,2_Binder_BetaSheet%,3_Binder_BetaSheet%,4_Binder_BetaSheet%,5_Binder_BetaSheet%,Average_Binder_Energy_Score,1_Binder_Energy_Score,2_Binder_Energy_Score,3_Binder_Energy_Score,4_Binder_Energy_Score,5_Binder_Energy_Score,Average_Binder_Helix%,1_Binder_Helix%,2_Binder_Helix%,3_Binder_Helix%,4_Binder_Helix%,5_Binder_Helix%,Average_Binder_Loop%,1_Binder_Loop%,2_Binder_Loop%,3_Binder_Loop%,4_Binder_Loop%,5_Binder_Loop%,Average_InterfaceAAs,1_InterfaceAAs,2_InterfaceAAs,3_InterfaceAAs,4_InterfaceAAs,5_InterfaceAAs,Average_InterfaceHbondsPercentage,1_InterfaceHbondsPercentage,2_InterfaceHbondsPercentage,3_InterfaceHbondsPercentage,4_InterfaceHbondsPercentage,5_InterfaceHbondsPercentage,Average_InterfaceUnsatHbondsPercentage,1_InterfaceUnsatHbondsPercentage,2_InterfaceUnsatHbondsPercentage,3_InterfaceUnsatHbondsPercentage,4_InterfaceUnsatHbondsPercentage,5_InterfaceUnsatHbondsPercentage,Average_Interface_BetaSheet%,1_Interface_BetaSheet%,2_Interface_BetaSheet%,3_Interface_BetaSheet%,4_Interface_BetaSheet%,5_Interface_BetaSheet%,Average_Interface_Helix%,1_Interface_Helix%,2_Interface_Helix%,3_Interface_Helix%,4_Interface_Helix%,5_Interface_Helix%,Average_Interface_Hydrophobicity,1_Interface_Hydrophobicity,2_Interface_Hydrophobicity,3_Interface_Hydrophobicity,4_Interface_Hydrophobicity,5_Interface_Hydrophobicity,Average_Interface_Loop%,1_Interface_Loop%,2_Interface_Loop%,3_Interface_Loop%,4_Interface_Loop%,5_Interface_Loop%,Average_Interface_SASA_%,1_Interface_SASA_%,2_Interface_SASA_%,3_Interface_SASA_%,4_Interface_SASA_%,5_Interface_SASA_%,Average_LIS,1_LIS,2_LIS,3_LIS,4_LIS,5_LIS,Average_PackStat,1_PackStat,2_PackStat,3_PackStat,4_PackStat,5_PackStat,Average_Relaxed_Clashes,1_Relaxed_Clashes,2_Relaxed_Clashes,3_Relaxed_Clashes,4_Relaxed_Clashes,5_Relaxed_Clashes,Average_ShapeComplementarity,1_ShapeComplementarity,2_ShapeComplementarity,3_ShapeComplementarity,4_ShapeComplementarity,5_ShapeComplementarity,Average_Surface_Hydrophobicity,1_Surface_Hydrophobicity,2_Surface_Hydrophobicity,3_Surface_Hydrophobicity,4_Surface_Hydrophobicity,5_Surface_Hydrophobicity,Average_Unrelaxed_Clashes,1_Unrelaxed_Clashes,2_Unrelaxed_Clashes,3_Unrelaxed_Clashes,4_Unrelaxed_Clashes,5_Unrelaxed_Clashes,Average_binder_monomer_RMSD,1_binder_monomer_RMSD,2_binder_monomer_RMSD,3_binder_monomer_RMSD,4_binder_monomer_RMSD,5_binder_monomer_RMSD,Average_chains_ptm_0,1_chains_ptm_0,2_chains_ptm_0,3_chains_ptm_0,4_chains_ptm_0,5_chains_ptm_0,Average_chains_ptm_1,1_chains_ptm_1,2_chains_ptm_1,3_chains_ptm_1,4_chains_ptm_1,5_chains_ptm_1,Average_complex_ipde,1_complex_ipde,2_complex_ipde,3_complex_ipde,4_complex_ipde,5_complex_ipde,Average_complex_iplddt,1_complex_iplddt,2_complex_iplddt,3_complex_iplddt,4_complex_iplddt,5_complex_iplddt,Average_complex_pde,1_complex_pde,2_complex_pde,3_complex_pde,4_complex_pde,5_complex_pde,Average_complex_plddt,1_complex_plddt,2_complex_plddt,3_complex_plddt,4_complex_plddt,5_complex_plddt,Average_confidence_score,1_confidence_score,2_confidence_score,3_confidence_score,4_confidence_score,5_confidence_score,Average_dG,1_dG,1_dG/dSASA,2_dG,2_dG/dSASA,3_dG,3_dG/dSASA,4_dG,4_dG/dSASA,5_dG,5_dG/dSASA,Average_dSASA,1_dSASA,2_dSASA,3_dSASA,4_dSASA,5_dSASA,Average_i_pLDDT,1_i_pLDDT,2_i_pLDDT,3_i_pLDDT,4_i_pLDDT,5_i_pLDDT,Average_ipSAE,1_ipSAE,2_ipSAE,3_ipSAE,4_ipSAE,5_ipSAE,Average_ipSAE_d0chn,1_ipSAE_d0chn,2_ipSAE_d0chn,3_ipSAE_d0chn,4_ipSAE_d0chn,5_ipSAE_d0chn,Average_ipSAE_d0dom,1_ipSAE_d0dom,2_ipSAE_d0dom,3_ipSAE_d0dom,4_ipSAE_d0dom,5_ipSAE_d0dom,Average_ipTM_af,1_ipTM_af,2_ipTM_af,3_ipTM_af,4_ipTM_af,5_ipTM_af,Average_ipTM_d0chn,1_ipTM_d0chn,2_ipTM_d0chn,3_ipTM_d0chn,4_ipTM_d0chn,5_ipTM_d0chn,Average_iptm,1_iptm,2_iptm,3_iptm,4_iptm,5_iptm,Average_ligand_iptm,1_ligand_iptm,2_ligand_iptm,3_ligand_iptm,4_ligand_iptm,5_ligand_iptm,Average_monomer_lDDT,1_monomer_lDDT,2_monomer_lDDT,3_monomer_lDDT,4_monomer_lDDT,5_monomer_lDDT,Average_n_InterfaceHbonds,1_n_InterfaceHbonds,2_n_In

### 4.5 Write the final binders to a directory

#### 4.5.1 Add binder paths back to the file

In [37]:
top_binders = boltz_bindcraft_style_df_filtered
top_binders_merged = top_binders.merge(combined_boltz_df[["binder_name","pdb_path"]].drop_duplicates("binder_name"),how="left",on="binder_name")
top_binders_merged = top_binders_merged.rename(columns={"chain_a_seq":"binder_sequence"})
top_binders_merged

,prediction_name,Average_Binder_BetaSheet%,1_Binder_BetaSheet%,2_Binder_BetaSheet%,3_Binder_BetaSheet%,4_Binder_BetaSheet%,5_Binder_BetaSheet%,Average_Binder_Energy_Score,1_Binder_Energy_Score,2_Binder_Energy_Score,3_Binder_Energy_Score,4_Binder_Energy_Score,5_Binder_Energy_Score,Average_Binder_Helix%,1_Binder_Helix%,2_Binder_Helix%,3_Binder_Helix%,4_Binder_Helix%,5_Binder_Helix%,Average_Binder_Loop%,1_Binder_Loop%,2_Binder_Loop%,3_Binder_Loop%,4_Binder_Loop%,5_Binder_Loop%,Average_InterfaceAAs,1_InterfaceAAs,2_InterfaceAAs,3_InterfaceAAs,4_InterfaceAAs,5_InterfaceAAs,Average_InterfaceHbondsPercentage,1_InterfaceHbondsPercentage,2_InterfaceHbondsPercentage,3_InterfaceHbondsPercentage,4_InterfaceHbondsPercentage,5_InterfaceHbondsPercentage,Average_InterfaceUnsatHbondsPercentage,1_InterfaceUnsatHbondsPercentage,2_InterfaceUnsatHbondsPercentage,3_InterfaceUnsatHbondsPercentage,4_InterfaceUnsatHbondsPercentage,5_InterfaceUnsatHbondsPercentage,Average_Interface_BetaSheet%,1_Interface_BetaSheet%,2_Interface_BetaSheet%,3_Interface_BetaSheet%,4_Interface_BetaSheet%,5_Interface_BetaSheet%,Average_Interface_Helix%,1_Interface_Helix%,2_Interface_Helix%,3_Interface_Helix%,4_Interface_Helix%,5_Interface_Helix%,Average_Interface_Hydrophobicity,1_Interface_Hydrophobicity,2_Interface_Hydrophobicity,3_Interface_Hydrophobicity,4_Interface_Hydrophobicity,5_Interface_Hydrophobicity,Average_Interface_Loop%,1_Interface_Loop%,2_Interface_Loop%,3_Interface_Loop%,4_Interface_Loop%,5_Interface_Loop%,Average_Interface_SASA_%,1_Interface_SASA_%,2_Interface_SASA_%,3_Interface_SASA_%,4_Interface_SASA_%,5_Interface_SASA_%,Average_LIS,1_LIS,2_LIS,3_LIS,4_LIS,5_LIS,Average_PackStat,1_PackStat,2_PackStat,3_PackStat,4_PackStat,5_PackStat,Average_Relaxed_Clashes,1_Relaxed_Clashes,2_Relaxed_Clashes,3_Relaxed_Clashes,4_Relaxed_Clashes,5_Relaxed_Clashes,Average_ShapeComplementarity,1_ShapeComplementarity,2_ShapeComplementarity,3_ShapeComplementarity,4_ShapeComplementarity,5_ShapeComplementarity,Average_Surface_Hydrophobicity,1_Surface_Hydrophobicity,2_Surface_Hydrophobicity,3_Surface_Hydrophobicity,4_Surface_Hydrophobicity,5_Surface_Hydrophobicity,Average_Unrelaxed_Clashes,1_Unrelaxed_Clashes,2_Unrelaxed_Clashes,3_Unrelaxed_Clashes,4_Unrelaxed_Clashes,5_Unrelaxed_Clashes,Average_binder_monomer_RMSD,1_binder_monomer_RMSD,2_binder_monomer_RMSD,3_binder_monomer_RMSD,4_binder_monomer_RMSD,5_binder_monomer_RMSD,Average_chains_ptm_0,1_chains_ptm_0,2_chains_ptm_0,3_chains_ptm_0,4_chains_ptm_0,5_chains_ptm_0,Average_chains_ptm_1,1_chains_ptm_1,2_chains_ptm_1,3_chains_ptm_1,4_chains_ptm_1,5_chains_ptm_1,Average_complex_ipde,1_complex_ipde,2_complex_ipde,3_complex_ipde,4_complex_ipde,5_complex_ipde,Average_complex_iplddt,1_complex_iplddt,2_complex_iplddt,3_complex_iplddt,4_complex_iplddt,5_complex_iplddt,Average_complex_pde,1_complex_pde,2_complex_pde,3_complex_pde,4_complex_pde,5_complex_pde,Average_complex_plddt,1_complex_plddt,2_complex_plddt,3_complex_plddt,4_complex_plddt,5_complex_plddt,Average_confidence_score,1_confidence_score,2_confidence_score,3_confidence_score,4_confidence_score,5_confidence_score,Average_dG,1_dG,1_dG/dSASA,2_dG,2_dG/dSASA,3_dG,3_dG/dSASA,4_dG,4_dG/dSASA,5_dG,5_dG/dSASA,Average_dSASA,1_dSASA,2_dSASA,3_dSASA,4_dSASA,5_dSASA,Average_i_pLDDT,1_i_pLDDT,2_i_pLDDT,3_i_pLDDT,4_i_pLDDT,5_i_pLDDT,Average_ipSAE,1_ipSAE,2_ipSAE,3_ipSAE,4_ipSAE,5_ipSAE,Average_ipSAE_d0chn,1_ipSAE_d0chn,2_ipSAE_d0chn,3_ipSAE_d0chn,4_ipSAE_d0chn,5_ipSAE_d0chn,Average_ipSAE_d0dom,1_ipSAE_d0dom,2_ipSAE_d0dom,3_ipSAE_d0dom,4_ipSAE_d0dom,5_ipSAE_d0dom,Average_ipTM_af,1_ipTM_af,2_ipTM_af,3_ipTM_af,4_ipTM_af,5_ipTM_af,Average_ipTM_d0chn,1_ipTM_d0chn,2_ipTM_d0chn,3_ipTM_d0chn,4_ipTM_d0chn,5_ipTM_d0chn,Average_iptm,1_iptm,2_iptm,3_iptm,4_iptm,5_iptm,Average_ligand_iptm,1_ligand_iptm,2_ligand_iptm,3_ligand_iptm,4_ligand_iptm,5_ligand_iptm,Average_monomer_lDDT,1_monomer_lDDT,2_monomer_lDDT,3_monomer_lDDT,4_monomer_lDDT,5_monomer_lDDT,Average_n_InterfaceHbonds,1_n_InterfaceHbonds,2_n_In

#### 4.5.2 Write the tsv file, pdb structures, and fastas into a directory

In [38]:
## Replace with your own output path
report_directory = "/data1/lareauc/users/chuh/softwares/CARPNN/examples/01_bindcraft_boltz_refolded/final_report"
report_name = "BCMABinders"


def write_report(data,report_dir,report_name="report",pdb_maps=None,id_col="barcode",seq_col="seq"):
    """
    Write report related entries like csv, pdbs, and pdf to the given directory
    pdb_maps are directory names plus the filepath column
    """
    os.makedirs(report_dir,exist_ok=True)
    ## TSV section
    report_tsv_path = os.path.join(report_dir,f"{report_name}.tsv")
    data.to_csv(report_tsv_path,sep="\t",index=False)
    ## PDB section
    if pdb_maps is not None:
        for dirname in pdb_maps:
            dest_dir = os.path.join(report_dir,dirname)
            os.makedirs(dest_dir,exist_ok=True)
            for idx,row in data.iterrows():
                pdb_path = row[pdb_maps[dirname]]
                if not os.path.exists(pdb_path):
                    print(f"Warning: {pdb_path} is not found")
                #print(f"Copying {pdb_path} to {dest_dir}")
                shutil.copy(pdb_path,dest_dir)
    ## FASTA section
    report_fasta_path = os.path.join(report_dir,f"{report_name}.fasta")
    df_to_fasta(data,id_col,seq_col,report_fasta_path)
    print("Done writing report")

def df_to_fasta(data,id_col,seq_col,out_file):
    """
    Convenience function to write 2 columns in dataframe into fasta file
    """
    with open(out_file, 'w') as fasta_file:
        for index, row in data.iterrows():
            to_write = f">{row[id_col]}\n{row[seq_col]}\n"
            fasta_file.write(to_write)

write_report(
    top_binders_merged,
    report_dir=report_directory,
    report_name=report_name,
    id_col="binder_name",
    seq_col="binder_sequence",
    pdb_maps={
        "boltz1_pdbs":"pdb_path"
    }
)

Done writing report
